In [1]:
import pandas as pd
import torch
import imageio
import os, glob
import matplotlib.pyplot as plt

In [2]:
csv_path = '/opt/ml/input/data/train/train_label_correct.csv'
train_csv = pd.read_csv(csv_path)
train_csv

,Unnamed: 0,id,gender,race,age,mask,img_path,class
0,0,1143,female,Asian,35,mask3.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
1,1,1143,female,Asian,35,incorrect_mask.jpg,/opt/ml/input/data/train/new_images/001143_fem...,10
2,2,1143,female,Asian,35,normal.jpg,/opt/ml/input/data/train/new_images/001143_fem...,16
3,3,1143,female,Asian,35,mask5.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
4,4,1143,female,Asian,35,mask1.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
...,...,...,...,...,...,...,...,...
18895,18895,5025,female,Asian,60,mask4.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18896,18896,5025,female,Asian,60,mask5.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18897,18897,5025,female,Asian,60,mask2.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18898,18898,5025,female,Asian,60,normal.jpg,/opt/ml/input/data/train/new_images/005025_fem...,17


In [4]:
gender_str2int_dict = {'male':0, 'female':1}
train_csv = train_csv.replace({'gender': gender_str2int_dict})
train_csv

,Unnamed: 0,id,gender,race,age,mask,img_path,class
0,0,1143,1,Asian,35,mask3.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
1,1,1143,1,Asian,35,incorrect_mask.jpg,/opt/ml/input/data/train/new_images/001143_fem...,10
2,2,1143,1,Asian,35,normal.jpg,/opt/ml/input/data/train/new_images/001143_fem...,16
3,3,1143,1,Asian,35,mask5.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
4,4,1143,1,Asian,35,mask1.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
...,...,...,...,...,...,...,...,...
18895,18895,5025,1,Asian,60,mask4.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18896,18896,5025,1,Asian,60,mask5.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18897,18897,5025,1,Asian,60,mask2.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18898,18898,5025,1,Asian,60,normal.jpg,/opt/ml/input/data/train/new_images/005025_fem...,17


In [5]:
def age_labelling(age_raw):
    age = int(age_raw)
    if age < 30:
        age = 0
    elif age < 60:
        age = 1
    else:
        age = 2
    return age

In [6]:
train_csv['age'] = train_csv['age'].apply(age_labelling)
train_csv

,Unnamed: 0,id,gender,race,age,mask,img_path,class
0,0,1143,1,Asian,1,mask3.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
1,1,1143,1,Asian,1,incorrect_mask.jpg,/opt/ml/input/data/train/new_images/001143_fem...,10
2,2,1143,1,Asian,1,normal.jpg,/opt/ml/input/data/train/new_images/001143_fem...,16
3,3,1143,1,Asian,1,mask5.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
4,4,1143,1,Asian,1,mask1.jpg,/opt/ml/input/data/train/new_images/001143_fem...,4
...,...,...,...,...,...,...,...,...
18895,18895,5025,1,Asian,2,mask4.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18896,18896,5025,1,Asian,2,mask5.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18897,18897,5025,1,Asian,2,mask2.jpg,/opt/ml/input/data/train/new_images/005025_fem...,5
18898,18898,5025,1,Asian,2,normal.jpg,/opt/ml/input/data/train/new_images/005025_fem...,17


In [7]:
def mask_labelling(mask_raw):
    if mask_raw.find('incorrect') != -1:
        mask = 1
    elif mask_raw.find('normal') != -1:
        mask = 2
    else:
        mask = 0
    return mask

In [8]:
train_csv['mask'] = train_csv['mask'].apply(mask_labelling)
train_csv

,Unnamed: 0,id,gender,race,age,mask,img_path,class
0,0,1143,1,Asian,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
1,1,1143,1,Asian,1,1,/opt/ml/input/data/train/new_images/001143_fem...,10
2,2,1143,1,Asian,1,2,/opt/ml/input/data/train/new_images/001143_fem...,16
3,3,1143,1,Asian,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
4,4,1143,1,Asian,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
...,...,...,...,...,...,...,...,...
18895,18895,5025,1,Asian,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18896,18896,5025,1,Asian,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18897,18897,5025,1,Asian,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18898,18898,5025,1,Asian,2,2,/opt/ml/input/data/train/new_images/005025_fem...,17


In [9]:
final_train = train_csv[['id', 'gender', 'age', 'mask', 'img_path', 'class']]

In [10]:
final_train

,id,gender,age,mask,img_path,class
0,1143,1,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
1,1143,1,1,1,/opt/ml/input/data/train/new_images/001143_fem...,10
2,1143,1,1,2,/opt/ml/input/data/train/new_images/001143_fem...,16
3,1143,1,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
4,1143,1,1,0,/opt/ml/input/data/train/new_images/001143_fem...,4
...,...,...,...,...,...,...
18895,5025,1,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18896,5025,1,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18897,5025,1,2,0,/opt/ml/input/data/train/new_images/005025_fem...,5
18898,5025,1,2,2,/opt/ml/input/data/train/new_images/005025_fem...,17


In [11]:
save_path = '/opt/ml/input/data/train/final_train.csv'
final_train.to_csv(save_path)